## Use Case and Better Evalution

1. Load in tokenizer and model

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("./bert-fake-news")
model.eval()  


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

2. Interpret results using LIME, output html file with explanation

In [19]:
import torch
import numpy as np
from lime.lime_text import LimeTextExplainer
import webbrowser

class_names = ["Fake", "Real"]

explainer = LimeTextExplainer(class_names=class_names)

sample_text = "Trump launches nuclear warhead on Iran and starts world war 3."

def predict_proba(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()
    return probs

probs = predict_proba([sample_text])[0]
predicted_class = np.argmax(probs)
confidence = probs[predicted_class]

print(f"Prediction: {class_names[predicted_class]} ({confidence*100:.2f}% confidence)")

explanation = explainer.explain_instance(
    sample_text,
    predict_proba,
    num_features=10,
    labels=(predicted_class,)
)

with open("lime_explanation.html", "w", encoding="utf-8") as f:
    f.write(explanation.as_html())

print("Explanation saved to lime_explanation.html.")
webbrowser.open("lime_explanation.html")

Prediction: Real (99.90% confidence)
Explanation saved to lime_explanation.html.


True

3. Set up prompt for gemini

In [20]:
lime_text = explanation.as_list(label=predicted_class)

explanation_summary = "\n".join(
    f"The word '{word}' contributed {weight:.2f} to the prediction."
    for word, weight in lime_text
)

llm_prompt = f"""
The model classified the article as '{class_names[predicted_class]}' with {confidence*100:.1f}% confidence.

Here are the top contributing words from the model's explanation:

{explanation_summary}

Can you explain in plain English why the model might have made this decision?
"""

4. Use gemini to transfer model results and LIME explanation to human readable text

In [23]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content([
    "You are an AI assistant that explains the results of a machine learning system which decides whether a news article is fake or real. "
    "Explain the results clearly and simply, so that anyone without technical knowledge can understand. "
    "Avoid mentioning the AI model or technical details. Do not explain the weights, write the text as if its an authority and not the model thinks this the model thinks that."
    "Add the confidence percentage at the end, this is very important"
    "Focus on helping people grasp why the system made its decision."
    "I don't want lists of the words, I want the explanation only!",
    llm_prompt
])

print(response.text)




This news article appears to be real because it contains a mix of words often found in factual reporting about international relations and events. The presence of terms like "warhead," "nuclear," and "Iran," combined with references to leaders like "Trump," suggests the article discusses potential geopolitical issues in a serious and grounded manner. While words like "launches" and "starts" might sometimes appear in sensationalized headlines, their impact is offset by the other terms that lend credibility to the article's overall tone. The inclusion of specific numbers like "3" also hints at a level of detail characteristic of genuine news reports.

Confidence: 99.9%

